# Brier Scores with varying ensemble member sizes

In [1]:
# general tools
import os
import sys
import time
import h5py
# import zarr
import numba as nb
from glob import glob

In [2]:
import numpy as np
from datetime import datetime, timedelta

sys.path.insert(0, '/glade/u/home/ksha/GAN_proj/')
sys.path.insert(0, '/glade/u/home/ksha/GAN_proj/libs/')

from namelist import *
import data_utils as du
import verif_utils as vu

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
def bs(y_true, y_pred):
    return np.nanmean((y_true - y_pred)**2)

In [6]:
with h5py.File(save_dir+'CCPA_domain.hdf', 'r') as h5io:
    lon_CCPA = h5io['lon_CCPA'][...]
    lat_CCPA = h5io['lat_CCPA'][...]
    land_mask_CCPA = h5io['land_mask_CCPA'][...]

land_mask = land_mask_CCPA > 0

In [13]:
filename = 'prob_by_ens_size3.hdf'
with h5py.File(result_dir+filename, 'r') as h5io:
    LDM_prob_p99 = h5io['LDM_prob_p99'][...]
    LDM_prob_40 = h5io['LDM_prob_40'][...]

LDM_prob_p99 = LDM_prob_p99[:, :, :, land_mask]
LDM_prob_40 = LDM_prob_40[:, :, :, land_mask]

In [8]:
with h5py.File(camp_dir+'CCPA/CCPA_lead_y2021.hdf', 'r') as h5io:
    CCPA_true = h5io['CCPA_lead'][...]

with h5py.File(camp_dir+'CCPA/CCPA_CDFs_lead_2002_2019.hdf', 'r') as h5io:
    CCPA_CDFs = h5io['CCPA_CDFs_lead'][...]
q99 = CCPA_CDFs[99, ...]

CCPA_true_40 = CCPA_true > 40
CCPA_true_40 = CCPA_true_40[:, :, land_mask]

CCPA_true_99 = CCPA_true > q99[None, ...]
CCPA_true_99 = CCPA_true_99[:, :, land_mask]

In [14]:
# hist_bins = np.linspace(0, 1, 19)

# y_true = CCPA_true_40[:, 0:1, :].ravel()
# y_pred = LDM_prob_40[1, :, 0:1, ...].ravel()

# output_bundle = vu.reliability_diagram_bootstrap(y_true, y_pred, N_boost=1, hist_bins=hist_bins)
# prob_true, prob_pred, hist_bins_, use_, o_bar, prob_pred_mean, prob_true_mean = output_bundle

# fake_x = np.linspace(0, 1, 10)
# plt.figure(figsize=(6, 6))

# plt.plot(prob_pred, prob_true, 'r-', lw=1)
# plt.plot(prob_pred_mean, prob_true_mean, 'k-', lw=3)

# plt.plot(fake_x, fake_x, 'k--')
# plt.xlim([0, 1])
# plt.ylim([0, 1])
# plt.xlabel('Predicted probability', fontsize=14)
# plt.ylabel('Observed relative frequency', fontsize=14)

In [15]:
BS_40_0_8 = np.empty(8)
BS_99_0_8 = np.empty(8)

BS_40_8_16 = np.empty(8)
BS_99_8_16 = np.empty(8)

BS_40_16_24 = np.empty(8)
BS_99_16_24 = np.empty(8)

for i in range(8):
    LDM_prob_40_ = LDM_prob_40[i, ...]
    LDM_prob_p99_ = LDM_prob_p99[i, ...]
    
    BS_40_0_8[i] = bs(CCPA_true_40[:, 0:8, :].ravel(), LDM_prob_40_[:, 0:8, :].ravel())
    BS_99_0_8[i] = bs(CCPA_true_99[:, 0:8, :].ravel(), LDM_prob_p99_[:, 0:8, :].ravel())

    BS_40_8_16[i] = bs(CCPA_true_40[:, 8:16, :].ravel(), LDM_prob_40_[:, 8:16, :].ravel())
    BS_99_8_16[i] = bs(CCPA_true_99[:, 8:16, :].ravel(), LDM_prob_p99_[:, 8:16, :].ravel())

    BS_40_16_24[i] = bs(CCPA_true_40[:, 16:24, :].ravel(), LDM_prob_40_[:, 16:24, :].ravel())
    BS_99_16_24[i] = bs(CCPA_true_99[:, 16:24, :].ravel(), LDM_prob_p99_[:, 16:24, :].ravel())

In [38]:
BS_40 = np.concatenate((BS_40_0_8[:, None], 
                        BS_40_8_16[:, None], 
                        BS_40_16_24[:, None]), axis=1)

In [41]:
BS_99 = np.concatenate((BS_99_0_8[:, None], 
                        BS_99_8_16[:, None], 
                        BS_99_16_24[:, None]), axis=1)

In [42]:
save_dict = {}
save_dict['BS_40'] = BS_40
save_dict['BS_99'] = BS_99
#np.save(result_dir+'ENS_num_verif.npy', save_dict, allow_pickle=True)

In [36]:
#plt.plot(BS_40_0_8_)
#plt.plot(BS_40_8_16_)
#plt.plot(BS_40_16_24_)

In [43]:
#plt.plot(BS_99_0_8_)
#plt.plot(BS_99_8_16_)
#plt.plot(BS_99_16_24_)